# Introduction

This notebook aims to present a very simple technique to perform image classification. Comparing to most of trending techniques, this one do not need a lot of materials, and computational power. So it can be run on almost any laptop. You will see that a ***70% accuracy*** can be reached very easily on the ***dog breed identification*** challenge.

This notebook is organized as follow : 
1. First some data exploration
2. Features extraction
3. Final predictions

# Data exploration

To experiment with this technique, I chose the **Dog Breed Identification challenge**. But a lot of Image Classification task could be adress in the same way.

First let's see what this dataset is all about.

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

## Loading data

In [ ]:
path_to_data = '../input/dog-breed-identification'
df = pd.read_csv(os.path.join(path_to_data,'labels.csv'))

## Breed distribution

Let's see the distribution of the different dog breed. 

For this I used pandas which comes with a lot of fancy functions for data exploration.

In [ ]:
# Build a dataframe with the number of instances in each class
breed_distrib = df['breed'].value_counts()
breed_distrib.columns = ['breed', 'number']

# Horizontal bar plot
plt.figure(figsize=(30,100))
sns.set(style="whitegrid")
sns.set(font_scale=5)
ax = sns.barplot(breed_distrib,breed_distrib.index)
plt.show()

**Good news**, the distribution is relatively balanced. The less represented breed contrains 65 images and the more represented contains 110 images.

**Bad news**, there isn't a lot of images per class. In this context we need to be careful to overfitting. 

## Image exploration

Now let's see what these images look like. And because I don't know myself every dog breed,  let's plot one image for each one ;)

In [ ]:
sns.set(font_scale=2)
n_breeds = len(breed_distrib.index)
print('Number of breeds : ', n_breeds)
for i in range(n_breeds):
    br = breed_distrib.index[i]
    path = df.loc[df['breed'] == br].iloc[0].id + '.jpg'
    path = os.path.join(path_to_data,'train',path)
    img = plt.imread(path)
    plt.axis('off')
    plt.imshow(img)
    plt.title(br)
    plt.show() 

Most of the images are centered on the dog but some of them are surrounded by their master or other objects which harden the task.

# Features Extraction

Most Computer Vision task are solved by using a very large and complicated Neural Network which require millons and millons of parameters to train. This can't be achieved by simple laptop, it requires one or more GPU and a lot of memory. Since I didn't have access to such materials I decided to use a pre-trained model to extract high level features for this task.

## Load pre-trained model

A lot of pre-trained model are available on most machine learning frameworks including PyTorch and Keras. 

Here I decided to use Keras and the VGG16 model trained on ImageNet dataset.

![](https://s3.ap-south-1.amazonaws.com/techleer/309.jpg)


VGG16 model is a very deep architecture which achieved great performance on ImageNet. 

But we don't want to perform fine-tuning because it required a lot of computational power. Instead we will cut the model at the first fully connected layer. 

With this new simplified model, we will pass the data throught the network. The ouput is then higher level and abstract features which should be useful to perform classification task.

In [ ]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
vgg16_weights = '../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
base_model = VGG16(weights=vgg16_weights)

In [ ]:
base_model.summary()

Since we do not want to perform fine-tuning because it is very computationally expensive, we will cut vgg16 at its first fully connected layer called 'fc1'.

In [ ]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## Generator

To avoid loading all images in memory at once, I used a generator. It enables to load only a batch of images.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


df['breed'] = LabelEncoder().fit_transform(df['breed'])
y = df['breed'] 
onehot = OneHotEncoder()
y = onehot.fit_transform(np.expand_dims(y, axis=1)).toarray()

#Generator
def generator(df):
    path_train = '../input/dog-breed-identification/train'
    while 1:
        for i in range(int(df.shape[0])):
            img_path = os.path.join(path_train, df.iloc[i]['id']+ '.jpg')
    
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            y = df.iloc[i]['breed']
            y = onehot.transform(y).toarray()
            #print(img.shape,np.array([y]).shape)
            yield (x,y)
                    
gen = generator(df)

## Extract features

In [ ]:
X_pred = model.predict_generator(gen,steps=10221, verbose=1)

# Final Predictions

With these new features, we can now run a simple classifier.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pred, df.iloc[:10221]['breed'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)

print("Incredible accuracy of : ",acc)

With this simple technique we reached around ***70 % of accuracy***.  It is a good score since there are ***120 different breeds***. 